<img src="assets/socalo-ICDA.png">

# Python para Finanzas y Ciencia de Datos
Federico Brun | fedejbrun@gmail.com

_Jueves 22 Octubre 2020_

## Programación Orientada a Objetos en Python

<img src="https://files.realpython.com/media/Object-Oriented-Programming-OOP-in-Python-3_Watermarked.0d29780806d5.jpg">

_Fuente: realpython.com_

Hasta ahora, venimos trabajando con el paradigma de la Programación Orientada a Objetos (OOP) en Python, pero sin haberlo abordado de manera directa.

De hecho, venimos trabajando con una mezcla de dos paradigmas sin haberlos abordado de manera directa: la OOP y la Programación Estructurada o Secuencial. 

En esta clase veremos las diferencias entre una y otra, profundizando con ejemplos que integren todo el estudio que ya hemos realizado orientado a la OOP.

## Paradigmas de Programación

Un paradigma de programación es un marco conceptual, un conjunto de ideas que describe una forma de entender la construcción de programa, como tal define:
* Las herramientas conceptuales que se pueden utilizar para construir un programa (objetos, relaciones, funciones, instrucciones).
* Las formas válidas de combinarlas.

Al ser Python un lenguaje de programación multipropósito, podemos implementar las herramientas conceptuales propias de cada paradigma.

Una definicion sencilla de cada uno de los paradigmas que hemos trabajado en el curso es la que sigue:

**Programación Estructurada**: Secuencia ordenada de instrucciones que puede bifurcar y/o interrumpir el flujo de ejecución del programa.

**Programación Orientada a Objetos**: En este modelo de paradigma se construyen modelos de objetos que representan elementos (objetos) del problema a resolver, que tienen características y funciones. 

## OOP en Python

Permite separar los diferentes componentes de un programa, simplificando así su creación, depuración y posteriores mejoras. La programación orientada a objetos disminuye los errores y promociona la reutilización del código. Es una manera especial de programar, que se acerca de alguna manera a cómo expresaríamos las cosas en la vida real.

En la OOP, definimos los siguientes componentes que tienen su correspondiente implementación en el lenguaje:
* Clases
* Objetos
* Atributos
* Métodos

En este paradigma, cobra importancia conceptos como _abstracción de datos, encapsulamiento , y modularización_ .

<div class="alert alert-block alert-info">
Una <b>clase</b> podría definirse como una plantilla para crear objetos.  Un <b>objeto</b> es una <i>instancia</i> de una <i>objeto</i> , con todos los atributos y métodos que actuan sobre esos atributos.</div>

Un ejemplo muy común para ilsutrar el concepto, consta de pensar una **clase** como un plano o un boceto o prototipo de un objeto por ejemplo un auto; y un **objeto** como los diferentes modelos de auto que se pueden producir a partir de ese plano.

A partir de eso, cada objeto de _tipo_ auto, tiene sus propios atributos y se comporta de una determinada manera.

<img src="./assets/objects1.png"/>

En otras palabras, el paradigma Orientado a Objetos consiste en un efoque para modelar una porcion de la realidad que nos interesa, definiendo "cosas" de la vida real, y la "relacion" entre estas cosas. 

Veamos cómo veníamos modelando la realidad, usando los tipos de objetos básicos de Python, como por ejemplo listas, Strings y números enteros.

In [6]:
alumno1 = ["Martin", "Gonzales", 19, 5234]
alumno2 = ["Antonia", "Lopez", 9876]

print(alumno1[0], alumno1[1])
print("Edad: " + str(alumno1[2]))
print("Nro Legajo: " + str(alumno1[3]))
print()
print(alumno2[0], alumno2[1])
print("Edad: " + str(alumno2[2]))
print("Nro Legajo: " + str(alumno2[3]))

Martin Gonzales
Edad: 19
Nro Legajo: 5234

Antonia Lopez
Edad: 9876


IndexError: list index out of range

Usando el enfoque anterior quedan en evidencia las limitaciones de modelar la relaidad usando tipos de datos que no estan pensados para esto. Para hacer que situaciones como la anterior mas manejables, usamos clases.

**Las clases o objetos sirven para definir estructuras de datos definidas por nosotros mismos.**

In [7]:
class Car:
    pass

Recordemos que dijimos que las clases tienen sus propias _características_ y _comportamientos_, o en terminos mas tecnicos, **atributos** y **metodos**.

<img src="./assets/class.png"/>

Existen muchas propiedades que podemos modelar de un auto. Una vez identificadas las que necesitamos para el sistema que estemos armando, debemos definirlas.

Para definirlas vamos a usar un metodo especial, llamado **constructor**.

In [25]:
class Car:
    def __init__(self, manufacturer, model, year, color, acceleration, brake):
        self.manufacturer = manufacturer
        self.model = model
        self.year = year 
        self.color = color 
        self.acceleration = acceleration
        self.brake = brake

Arriba, acabamos de definir el constructor, es decir, el metodo que va a hacer que la clase exista, con los parámetros que le especificamos a nuestra medida.

Los atributos creados en `__init__()` se llaman **atributos de instancia** porque seran propios de cada instancia una vez sean creadas.

<img src="./assets/objetos.png"/>

<div class="alert alert-block alert-info">
El proceso de crear un <b>objeto</b> a partir de una <b>clase</b> se denomina <b>Instanciación</b>.</div>

In [9]:
class Car:
    pass

In [10]:
sport_car = Car()
classic_car = Car()

In [11]:
sport_car

In [12]:
classic_car

In [13]:
sport_car == classic_car

False

A pesar de que `sport_car` y `classic_car` son **instancias** de la clase `Car` , representan dos **objetos** diferentes. 

Ahora vamos a instanciar los objetos nuevamente, pero pasando atributos al constructor.

In [15]:
class Car:
    def __init__(self, manufacturer, model, year, color, acceleration, brake):
        self.manufacturer = manufacturer
        self.model = model
        self.year = year 
        self.color = color 
        self.acceleration = acceleration
        self.brake = brake

In [16]:
sport_car = Car("Ferrari", "Portofino", 2020, "Rojo", 320, 5)
classic_car = Car("Ford", "T", 1927, "Negro", 71, 10 )

Dónde quedo el parámetro `self`?

Cuando instanciamos un objeto de tipo `Car` , Python crea una nueva instancia y la pasa como primer parámetro al constructor, por lo que no debemos preocuparnos por `self`. 

Despues de haber creado los dos objetos de tipo `Car` , podemos acceder a sus atributos utilizando `.` .

In [17]:
sport_car.manufacturer

'Ferrari'

In [18]:
sport_car.model

'Portofino'

In [19]:
classic_car.year

1927

In [20]:
classic_car.color

'Negro'

Podemos cambiar los atributos de un objeto, de forma dinámica, luego de haberlo creado. Por ejemplo si pintamos nuestra ferrari de color azul:

In [21]:
sport_car.color = "Azul"

In [22]:
sport_car.color

'Azul'

Hasta ahora definimos los atributos, nos queda definir el comportamiento de neustros objetos, o sus **metodos**:

<img src="./assets/class.png"/>

In [45]:
class Car:
    def __init__(self, manufacturer, model, year, color, acceleration, brake):
        self.manufacturer = manufacturer
        self.model = model
        self.year = year 
        self.color = color 
        self.acceleration = acceleration
        self.brake = brake
        
    def accelerate(self, speed):
        print("Acelerando hasta los " + str(speed) + " km/h")
        
    def deaccelerate(self, speed):
        print("Frenando hasta los " + str(speed) + " km/h")
        
    def print_car(self):
        print("Manufacturer: " + self.manufacturer
              + "\nModel: " + self.model 
              + "\nYear: " + str(self.year) 
              + "\nColor: " + self.color 
              + "\nAcceleration: " + str(self.acceleration) 
              + "\nBrake: " + str(self.brake)
             )

In [46]:
sport_car = Car("Ferrari", "Portofino", 2020, "Rojo", 320, 5)
classic_car = Car("Ford", "T", 1927, "Negro", 71, 10 )

In [47]:
sport_car.print_car()

Manufacturer: Ferrari
Model: Portofino
Year: 2020
Color: Rojo
Acceleration: 320
Brake: 5


In [48]:
classic_car.print_car()

Manufacturer: Ford
Model: T
Year: 1927
Color: Negro
Acceleration: 71
Brake: 10


In [49]:
sport_car.accelerate(250)

Acelerando hasta los 250 km/h


In [50]:
classic_car.deaccelerate(10)

Frenando hasta los 10 km/h
